In [12]:
# Import modules.
import torch
import random

import numpy as np

from game import Game
from network import H_GO
from torch import nn, optim
from torchsummary import summary
from dataReader import DataReader
from trainer import SL_Trainer, RL_Trainer

In [13]:
# Set training device.
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available.")
else:
    device = torch.device("cpu")
    print("CUDA is not available, using CPU instead.")

CUDA is available.


In [14]:
# Set random seed.
random_seed = 0

random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [15]:
# Set hyperparameters.
epochs = 5

batch_size = 256
data_num = 10
data_augment = False

In [16]:
# Load data.
data_reader = DataReader(dir_path="../dataset/pro", load_num=data_num, train_ratio=0.8, augment=data_augment)

Read raw data completed. (0.03 s)
Clear data completed. (0.00 s)
Convert data completed. (0.00 s)
Make train data completed. (0.01 s)
Split train and test data completed. (1744 train data, 437 test data).


In [17]:
# set model.
model = H_GO(input_size=8, output_size=361, hidden_dim=64)
model.to(device=device)

H_GO(
  (conv1): Conv2d(8, 64, kernel_size=(3, 3), stride=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (tanh): Tanh()
  (layers): Sequential(
    (0): Block(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (1): Block(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    

In [18]:
# Print model summary.
summary(model, input_size=(8, 19, 19))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 17, 17]           4,672
       BatchNorm2d-2           [-1, 64, 17, 17]             128
              Tanh-3           [-1, 64, 17, 17]               0
            Conv2d-4           [-1, 64, 17, 17]          36,928
       BatchNorm2d-5           [-1, 64, 17, 17]             128
              ReLU-6           [-1, 64, 17, 17]               0
            Conv2d-7           [-1, 64, 17, 17]          36,928
       BatchNorm2d-8           [-1, 64, 17, 17]             128
              ReLU-9           [-1, 64, 17, 17]               0
            Block-10           [-1, 64, 17, 17]               0
           Conv2d-11           [-1, 64, 17, 17]          36,928
      BatchNorm2d-12           [-1, 64, 17, 17]             128
             ReLU-13           [-1, 64, 17, 17]               0
           Conv2d-14           [-1, 64,

In [19]:
# Set optimizer and loss function.
optimizer = optim.RAdam(params=model.parameters())
loss_fu_policy = nn.CrossEntropyLoss()
loss_fu_value = nn.BCELoss()

In [20]:
sl_trainer = SL_Trainer(epochs=epochs, model=model, device=device, optimizer=optimizer, loss_fu_policy=loss_fu_policy, loss_fu_value=loss_fu_value)
sl_trainer.train(data_reader=data_reader, batch_size=batch_size)

Epoch: 000 | Loss of model: 39.887 | Accuracy of policy: 0.344% | Accuracy of value: 45.642%
Epoch: 001 | Loss of model: 39.206 | Accuracy of policy: 1.032% | Accuracy of value: 50.975%
Epoch: 002 | Loss of model: 37.810 | Accuracy of policy: 2.122% | Accuracy of value: 76.089%
Epoch: 003 | Loss of model: 36.487 | Accuracy of policy: 4.472% | Accuracy of value: 83.888%
Epoch: 004 | Loss of model: 35.058 | Accuracy of policy: 6.422% | Accuracy of value: 85.665%


In [21]:
sl_trainer.test(data_reader=data_reader, batch_size=batch_size)

Accuracy of policy: 0.000% | Accuracy of value: 25.400%


In [22]:
game = Game(size=19, model=model, device=device)
game.play()

AI's move: (15, 6)
  A B C D E F G H I J K L M N O P Q R S
A · · · · · · · · · · · · · · · · · · · 
B · · · · · · · · · · · · · · · · · · · 
C · · · · · · · · · · · · · · · · · · · 
D · · · · · · · · · · · · · · · · · · · 
E · · · · · · · · · · · · · · · · · · · 
F · · · · · · · · · · · · · · · · · · · 
G · · · · · · · · · · · · · · · · · · · 
H · · · · · · · · · · · · · · · · · · · 
I · · · · · · · · · · · · · · · · · · · 
J · · · · · · · · · · · · · · · · · · · 
K · · · · · · · · · · · · · · · · · · · 
L · · · · · · · · · · · · · · · · · · · 
M · · · · · · · · · · · · · · · · · · · 
N · · · · · · · · · · · · · · · · · · · 
O · · · · · · · · · · · · · · · · · · · 
P · · · · · ·(X)· · · · · · · · · · · · 
Q · · · · · · · · · · · · · · · · · · · 
R · · · · · · · · · · · · · · · · · · · 
S · · · · · · · · · · · · · · · · · · · 

Player's move: (1, 2)
  A B C D E F G H I J K L M N O P Q R S
A · · · · · · · · · · · · · · · · · · · 
B · ·(O)· · · · · · · · · · · · · · · · 
C · · · · · · · ·